In [1]:
import pandas as pd
import polars as pl
from pyproj import Geod

/var/folders/q1/9phfzjcx3qz5yclxc2kj3bmc0000gn/T/ipykernel_40368/3683528684.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Cleaning the wildfire data

In [3]:
wildfires_with_distances = pd.read_csv("generated_files/intermediate/wildfires_with_distances.csv")

# find wildfires within 650 miles from the city of interest
wildfires_in_proximity =  wildfires_with_distances[ wildfires_with_distances["Closest_Distance_Miles"] <= 650 ]

wildfires_in_proximity

/var/folders/q1/9phfzjcx3qz5yclxc2kj3bmc0000gn/T/ipykernel_40368/546716935.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  wildfires_with_distances = pd.read_csv("generated_files/intermediate/wildfires_with_distances.csv")


,OBJECTID,USGS_Assigned_ID,Assigned_Fire_Type,Fire_Year,Fire_Polygon_Tier,Fire_Attribute_Tiers,GIS_Acres,GIS_Hectares,Source_Datasets,Listed_Fire_Types,...,Listed_Rx_Reported_Acres,Listed_Map_Digitize_Methods,Wildfire_and_Rx_Flag,Overlap_Within_1_or_2_Flag,Circleness_Scale,Circle_Flag,Exclude_From_Summary_Rasters,Shape_Length,Shape_Area,Closest_Distance_Miles
0,14600,14600,Wildfire,1964,1,"1 (1), 3 (3)",65338.877636,26441.705659,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (3), Likely Wildfire (1)",...,NaN,NaN,NaN,NaN,0.263753,NaN,No,112240.801495,2.644171e+08,250.778640
1,14601,14601,Wildfire,1964,1,"1 (2), 3 (3), 5 (1)",55960.694158,22646.489455,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (5), Likely Wildfire (1)",...,NaN,NaN,NaN,NaN,0.224592,NaN,No,112566.141954,2.264649e+08,76.420104
2,14602,14602,Wildfire,1964,1,"1 (2), 3 (3)",19218.105903,7777.291530,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (4), Likely Wildfire (1)",...,NaN,NaN,NaN,NaN,0.138493,NaN,No,84004.974692,7.777292e+07,327.916719
3,14603,14603,Wildfire,1964,1,"1 (2), 3 (3)",18712.494746,7572.677954,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (4), Likely Wildfire (1)",...,NaN,NaN,NaN,NaN,0.391960,NaN,No,49273.004457,7.572678e+07,198.853859
4,14604,14604,Wildfire,1964,1,"1 (4), 3 (6)",16887.001024,6833.926855,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (8), Likely Wildfire (2)",...,NaN,NaN,NaN,NaN,0.392989,NaN,No,46746.577459,6.833927e+07,219.778526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117157,135056,135056,Prescribed Fire,2020,8,8 (2),14.545208,5.886237,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (2),...,NaN,NaN,NaN,NaN,0.299250,NaN,No,1572.196739,5.886237e+04,453.342716
117158,135057,135057,Prescribed Fire,2020,8,8 (3),16.412148,6.641761,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (3),...,NaN,NaN,NaN,"Caution, this Prescribed Fire in 2020 overlaps...",0.177425,NaN,No,2168.900740,6.641761e+04,406.632430
117159,135058,135058,Prescribed Fire,2020,8,8 (1),7.050837,2.853373,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (1),...,NaN,NaN,NaN,"Caution, this Prescribed Fire in 2020 overlaps...",0.374368,NaN,No,978.666221,2.853373e+04,449.230338
117160,135059,135059,Prescribed Fire,2020,8,8 (4),9.342668,3.780843,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (4),...,NaN,NaN,NaN,"Caution, this Prescribed Fire in 2020 overlaps...",0.123888,NaN,No,1958.326660,3.780843e+04,448.331439


In [4]:
# Cleaning the gaseous and particulate data
gaseous_data = pl.from_dataframe( pd.read_csv("generated_files/intermediate/gaseous_AQI_1964-2024.csv") )
# print(gaseous_data)

particulate_data = pl.from_dataframe( pd.read_csv("generated_files/intermediate/particulate_AQI_1964-2024.csv") )
# print(particulate_data)

# merging both gaseous and particle data
aqi_data = pl.concat([gaseous_data, particulate_data])
print(aqi_data)
print(aqi_data.columns)

shape: (218_695, 32)
┌───────────┬───────────┬───────────┬───────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ state_cod ┆ county_co ┆ site_numb ┆ parameter ┆ … ┆ city     ┆ cbsa_code ┆ cbsa      ┆ date_of_l │
│ e         ┆ de        ┆ er        ┆ _code     ┆   ┆ ---      ┆ ---       ┆ ---       ┆ ast_chang │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ str      ┆ i64       ┆ str       ┆ e         │
│ i64       ┆ i64       ┆ i64       ┆ i64       ┆   ┆          ┆           ┆           ┆ ---       │
│           ┆           ┆           ┆           ┆   ┆          ┆           ┆           ┆ str       │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ 6         ┆ 77        ┆ 1002      ┆ 42101     ┆ … ┆ Stockton ┆ 44700     ┆ Stockton- ┆ 2016-03-3 │
│           ┆           ┆           ┆           ┆   ┆          ┆           ┆ Lodi, CA  ┆ 1         │
│ 6         ┆ 77        ┆ 1002      ┆ 42101     ┆ … ┆ Stockton ┆ 44700

In [5]:
print("Unique stations - -")

unique_stations = (
    aqi_data
    .select(
        [
            'state_code',
            'county_code',
            'site_number',
            'latitude',
            'longitude'
        ]
    )
    .unique()
)
print(unique_stations)

Unique stations - -
shape: (13, 5)
┌────────────┬─────────────┬─────────────┬───────────┬─────────────┐
│ state_code ┆ county_code ┆ site_number ┆ latitude  ┆ longitude   │
│ ---        ┆ ---         ┆ ---         ┆ ---       ┆ ---         │
│ i64        ┆ i64         ┆ i64         ┆ f64       ┆ f64         │
╞════════════╪═════════════╪═════════════╪═══════════╪═════════════╡
│ 6          ┆ 77          ┆ 3003        ┆ 37.737985 ┆ -121.535503 │
│ 6          ┆ 77          ┆ 1003        ┆ 37.961578 ┆ -121.281414 │
│ 6          ┆ 77          ┆ 9           ┆ 37.906869 ┆ -121.148275 │
│ 6          ┆ 77          ┆ 4           ┆ 37.954075 ┆ -121.289634 │
│ 6          ┆ 77          ┆ 5           ┆ 37.84826  ┆ -121.446892 │
│ …          ┆ …           ┆ …           ┆ …         ┆ …           │
│ 6          ┆ 77          ┆ 3010        ┆ 38.029626 ┆ -121.354026 │
│ 6          ┆ 77          ┆ 3002        ┆ 37.739929 ┆ -121.533002 │
│ 6          ┆ 77          ┆ 3005        ┆ 37.682635 ┆ -121.442495 │

In [6]:
# calculate distances for each station from the city

geodcalc = Geod(ellps='WGS84')
CITY_LOCATION = {
    'city'   : 'Stockton',
    'latlon' : [37.975556, -121.300833] 
}


def distance_in_miles(row):
    d = geodcalc.inv(CITY_LOCATION["latlon"][1],CITY_LOCATION["latlon"][0],row["longitude"],row["latitude"])
    # convert the distance to miles
    distance_in_miles = d[2]*0.00062137
    return distance_in_miles

unique_stations_dist = (
    unique_stations
    .with_columns(
        pl.struct(["latitude", "longitude"])
        .map_elements(
            distance_in_miles,
            return_dtype=pl.Float64
        ).alias("distance_in_miles")
    )
    .sort("distance_in_miles")
)

print(unique_stations_dist)

shape: (13, 6)
┌────────────┬─────────────┬─────────────┬───────────┬─────────────┬───────────────────┐
│ state_code ┆ county_code ┆ site_number ┆ latitude  ┆ longitude   ┆ distance_in_miles │
│ ---        ┆ ---         ┆ ---         ┆ ---       ┆ ---         ┆ ---               │
│ i64        ┆ i64         ┆ i64         ┆ f64       ┆ f64         ┆ f64               │
╞════════════╪═════════════╪═════════════╪═══════════╪═════════════╪═══════════════════╡
│ 6          ┆ 77          ┆ 8           ┆ 37.995201 ┆ -121.309115 ┆ 1.428345          │
│ 6          ┆ 77          ┆ 1003        ┆ 37.961578 ┆ -121.281414 ┆ 1.433032          │
│ 6          ┆ 77          ┆ 4           ┆ 37.954075 ┆ -121.289634 ┆ 1.602768          │
│ 6          ┆ 77          ┆ 1002        ┆ 37.950741 ┆ -121.268523 ┆ 2.457991          │
│ 6          ┆ 77          ┆ 3010        ┆ 38.029626 ┆ -121.354026 ┆ 4.725914          │
│ …          ┆ …           ┆ …           ┆ …         ┆ …           ┆ …                 │
│ 6   

In [10]:
# find the mean aqi per site per gas per day
aqi_mean_granular = (
    aqi_data
    .filter([
        # filter only for valid data
        pl.col("validity_indicator") == "Y", # ~4k records have validity indicator as "N"
        ~pl.col("aqi").is_nan() # ~50k records do not have AQI info (.i.e 20%)
    ])
    .select(
        # select relevant features
        [
            'site_number',
            'parameter',
            'date_local',
            'aqi'
        ]
    )
    .group_by( 
        [
            'site_number',
            'date_local',
            'parameter'
        ]
    )
    .agg(pl.mean("aqi"))
)
print(aqi_mean_granular)

shape: (164_277, 32)
┌───────────┬───────────┬───────────┬───────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ state_cod ┆ county_co ┆ site_numb ┆ parameter ┆ … ┆ city     ┆ cbsa_code ┆ cbsa      ┆ date_of_l │
│ e         ┆ de        ┆ er        ┆ _code     ┆   ┆ ---      ┆ ---       ┆ ---       ┆ ast_chang │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ str      ┆ i64       ┆ str       ┆ e         │
│ i64       ┆ i64       ┆ i64       ┆ i64       ┆   ┆          ┆           ┆           ┆ ---       │
│           ┆           ┆           ┆           ┆   ┆          ┆           ┆           ┆ str       │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ 6         ┆ 77        ┆ 1002      ┆ 42101     ┆ … ┆ Stockton ┆ 44700     ┆ Stockton- ┆ 2023-02-0 │
│           ┆           ┆           ┆           ┆   ┆          ┆           ┆ Lodi, CA  ┆ 5         │
│ 6         ┆ 77        ┆ 1002      ┆ 42101     ┆ … ┆ Stockton ┆ 44700

In [11]:
# calculate the max aqi recorded at a station on a given day
# parameter corresponds to the gas that constituted to max aqi

# daily AQI is given by the formula as max of all pollutants' AQI (ref chatgpt)

aqi_max_per_station = (
    aqi_mean_granular
    .group_by(
        [
            'site_number',
            'date_local'
        ]
    )
    .agg(
        pl.max('aqi'),
    )
)
aqi_max_per_station

site_number,date_local,aqi
i64,str,f64
2010,"""2016-07-28""",44.0
9,"""1990-07-04""",44.0
1002,"""1968-08-10""",16.0
3003,"""1999-10-14""",87.0
2010,"""2011-09-14""",61.0
…,…,…
1002,"""1989-09-11""",38.0
3003,"""1998-07-02""",31.0
3005,"""2015-06-23""",74.0


In [12]:
# find yearly AQI from each station by taking average

aqi_yearly_per_station = (
    aqi_max_per_station
    .with_columns(
        pl.col("date_local").str.strptime(pl.Date, "%Y-%m-%d").dt.year().alias("year")
    )
    .group_by(
        [
            "site_number",
            "year"
        ]
    )
    .agg(
        pl.mean("aqi")
    )
)

print(aqi_yearly_per_station)

shape: (164, 3)
┌─────────────┬──────┬───────────┐
│ site_number ┆ year ┆ aqi       │
│ ---         ┆ ---  ┆ ---       │
│ i64         ┆ i32  ┆ f64       │
╞═════════════╪══════╪═══════════╡
│ 8           ┆ 1994 ┆ 9.360465  │
│ 9           ┆ 1990 ┆ 62.214286 │
│ 2010        ┆ 2015 ┆ 45.021978 │
│ 1002        ┆ 2000 ┆ 48.358696 │
│ 1002        ┆ 1980 ┆ 57.119565 │
│ …           ┆ …    ┆ …         │
│ 3003        ┆ 2004 ┆ 57.118421 │
│ 3005        ┆ 2020 ┆ 58.065574 │
│ 9           ┆ 2001 ┆ 53.485549 │
│ 5           ┆ 1980 ┆ 51.0      │
│ 1002        ┆ 1998 ┆ 56.934783 │
└─────────────┴──────┴───────────┘


In [13]:
# join  this table with the unique_station table to map the distances
yearly_aqi_weighted_avg = (
    aqi_yearly_per_station
    .join(
        unique_stations_dist,
        on = "site_number"
    )
    .drop(
        [
            "state_code",
            "county_code",
            "latitude",
            "longitude"
        ]
    )
    .with_columns(
        (1 / pl.col("distance_in_miles")).alias("weight")  # Invert distance to create weight
    ) 
    .with_columns(
        (pl.col("aqi") * pl.col("weight")).alias("weighted_aqi")  # AQI * weight
    ) 
    .group_by("year")
    .agg(
        [
          (pl.sum("weighted_aqi") / pl.sum("weight")).round(2).alias("weighted_avg_aqi")  # Weighted average AQI
        ]
    )
)
yearly_aqi_weighted_avg = yearly_aqi_weighted_avg.sort(by="year")
print(yearly_aqi_weighted_avg)

shape: (61, 2)
┌──────┬──────────────────┐
│ year ┆ weighted_avg_aqi │
│ ---  ┆ ---              │
│ i32  ┆ f64              │
╞══════╪══════════════════╡
│ 1964 ┆ 32.41            │
│ 1965 ┆ 25.73            │
│ 1966 ┆ 21.06            │
│ 1967 ┆ 26.93            │
│ 1968 ┆ 25.57            │
│ …    ┆ …                │
│ 2020 ┆ 59.68            │
│ 2021 ┆ 52.98            │
│ 2022 ┆ 43.13            │
│ 2023 ┆ 45.11            │
│ 2024 ┆ 38.37            │
└──────┴──────────────────┘


In [14]:
yearly_aqi_weighted_avg.write_csv("generated_files/intermediate/yearly_weighted_aqi_1964-2024.csv")